In [5]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("fr_core_news_lg")


In [6]:
def load_t_matcher(patterns, nlp):
    
    matcher = Matcher(nlp.vocab)
    [matcher.add(rule, [pattern]) for rule, pattern in patterns.items()] 
    
    return matcher


def run_baseline_fr(nlp, doc, t_matcher):
    
    appreciation = False
    patterns = []
    sequences = []
    
    praise = []
    negated = ["me remercie", "seigneur"]
    request = ["pouvez-vous", "pourriez-vous", "peux-tu", "est-il possible" , "serait-il possible", "merci d'avance"]
    
    t_matches = t_matcher(doc)
    
    request_match = []
    if t_matches:
        appreciation = True
        for match_id, start, end in t_matches:
            span = doc[start:end]
            # discard the match if captured sequence is actually a request-Gratitude (e.g. Can you email me the file please? Thanks)
            first_part = doc[:start].text.lower()
            request_match = [x for x in request if x in first_part]
            patterns.append(nlp.vocab.strings[match_id])
            sequences.append(span.text)
    
    for text in praise:
      if text in doc.text.lower():
        appreciation = True
    
    
    for text in negated:
      if text in doc.text.lower():
        appreciation = False
    
    if request_match:
      appreciation = False
    
    
    if appreciation:
      return appreciation, patterns, sequences
    else:
      return appreciation, [], []

In [7]:
patterns = {
    "je/nous/on - tenir - à - vous/te/les/le/la - remercier": [{"LOWER": {"IN": ["je", "on", "nous"]}}, {"LEMMA": "tenir"}, {"LOWER": "à"},  {"POS": {"IN": ["PRON", "DET"]}}, {"LEMMA":"remercier"}],
    "je/nous/on - tenir - à - remercier": [{"LOWER": {"IN": ["je", "on", "nous"]}}, {"LEMMA": "tenir"}, {"LOWER": "à"}, {"LEMMA":"remercier"}],
    "remerciement - adressé - à": [{"LEMMA": {"IN": ["remerciement", "gratitude"]}}, {"LEMMA": "adresser"}, {"LOWER": "à"}],
    "nos/mes - sincères/... - remerciments/...": [{"LOWER": {"IN": ["notre", "mon", "nos", "mes", "ma"]}}, {"LOWER": {"IN": ["sincères", "sincère"]}}, {"LOWER": {"IN": ["gratitude", "remerciement", "remerciements" ]}}]
}

In [8]:
texts = [
    "En cette belle journée passée au royaume magique, je tiens à remercier mille fois Brocéliande-Nausicaa et Cédric de la Tour de la Terreur",
    "Vous souvenez-vous quand vous avez rejoint Twitter ? Moi, oui ! MonAnniversaireTwitter Que le temps passe vite. 😭😭 je tiens à vous remercier pour le soutien.",
    "Nous tenons à remercier USER pour sa contribution. Sans son soutien, nous ne pourrions pas garantir que le Canada dispose d'une stratégie de dépistage et de sensibilisation par et pour la communauté.", 
    "Remerciements adressés à USER , USER et USER par tous ceux qui ont pris la parole, du représentant du site Kinyinya II au Conseiller du Gouverneur",
    "Nos sincères remerciements aux agents."
]

In [9]:
# load matchers
t_mat = load_t_matcher(patterns, nlp)

# process documents
docs = list(nlp.pipe(texts))


# inspect relevant token attributes
for doc in docs:
    print(doc.text)
    for t in doc:
        print("Token:", t.text, "\tPOS: ", t.tag_, "\tLemma: ", t.lemma_)
    print("__________________________________________________________________")


En cette belle journée passée au royaume magique, je tiens à remercier mille fois Brocéliande-Nausicaa et Cédric de la Tour de la Terreur
Token: En 	POS:  ADP 	Lemma:  en
Token: cette 	POS:  DET 	Lemma:  ce
Token: belle 	POS:  ADJ 	Lemma:  bel
Token: journée 	POS:  NOUN 	Lemma:  journée
Token: passée 	POS:  VERB 	Lemma:  passer
Token: au 	POS:  ADP 	Lemma:  au
Token: royaume 	POS:  NOUN 	Lemma:  royaume
Token: magique 	POS:  ADJ 	Lemma:  magique
Token: , 	POS:  PUNCT 	Lemma:  ,
Token: je 	POS:  PRON 	Lemma:  je
Token: tiens 	POS:  VERB 	Lemma:  tenir
Token: à 	POS:  ADP 	Lemma:  à
Token: remercier 	POS:  VERB 	Lemma:  remercier
Token: mille 	POS:  NUM 	Lemma:  mille
Token: fois 	POS:  NOUN 	Lemma:  fois
Token: Brocéliande 	POS:  ADJ 	Lemma:  brocéliand
Token: - 	POS:  PUNCT 	Lemma:  -
Token: Nausicaa 	POS:  NOUN 	Lemma:  nausicaa
Token: et 	POS:  CCONJ 	Lemma:  et
Token: Cédric 	POS:  PROPN 	Lemma:  Cédric
Token: de 	POS:  ADP 	Lemma:  de
Token: la 	POS:  DET 	Lemma:  le
Token: Tour 	P

In [10]:
# check predictions
for doc in docs:
    print("text:\t", doc.text)    
    appre, pat, seq = run_baseline_fr(nlp, doc, t_mat)
    if appre:
        print("This document contains gratitude.")
        print("Captured patterns:\t", pat)
        print("Evidence:\t", seq)
        print("_______________________________________")
        
    else:
        print("This document contains no gratitude.")
        print("_______________________________________")


text:	 En cette belle journée passée au royaume magique, je tiens à remercier mille fois Brocéliande-Nausicaa et Cédric de la Tour de la Terreur
This document contains gratitude.
Captured patterns:	 ['je/nous/on - tenir - à - remercier']
Evidence:	 ['je tiens à remercier']
_______________________________________
text:	 Vous souvenez-vous quand vous avez rejoint Twitter ? Moi, oui ! MonAnniversaireTwitter Que le temps passe vite. 😭😭 je tiens à vous remercier pour le soutien.
This document contains gratitude.
Captured patterns:	 ['je/nous/on - tenir - à - vous/te/les/le/la - remercier']
Evidence:	 ['je tiens à vous remercier']
_______________________________________
text:	 Nous tenons à remercier USER pour sa contribution. Sans son soutien, nous ne pourrions pas garantir que le Canada dispose d'une stratégie de dépistage et de sensibilisation par et pour la communauté.
This document contains gratitude.
Captured patterns:	 ['je/nous/on - tenir - à - remercier']
Evidence:	 ['Nous tenons à 